# TASK 1 - Data Cleaning and Preprocessing

In [22]:
# Customer Personality Analysis - Data Cleaning

import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("marketing_campaign.csv", sep='\t')

In [23]:
# Basic Info
print("Shape:", df.shape)
print(df.head())
print(df.info())

Shape: (2240, 29)
     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  04-09-2012       58       635  ...                  7             0   
1  08-03-2014       38        11  ...                  5             0   
2  21-08-2013       26       426  ...                  4             0   
3  10-02-2014       26        11  ...                  6             0   
4  19-01-2014       94       173  ...                  5             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain 

In [24]:
# Check missing values
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64


In [25]:
# Handle missing values
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(include='object').columns

for col in num_cols:
    df.loc[:, col] = df[col].fillna(df[col].median())

for col in cat_cols:
    df.loc[:, col] = df[col].fillna(df[col].mode()[0])

In [26]:
# Remove duplicates
print("\nDuplicates before:", df.duplicated().sum())
df.drop_duplicates(inplace=True)
print("Duplicates after:", df.duplicated().sum())


Duplicates before: 0
Duplicates after: 0


In [27]:
# Clean column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(' ', '_')
      .str.replace('[^0-9a-zA-Z_]', '', regex=True)
)

In [28]:
# Clean text columns
cat_cols = df.select_dtypes(include='object').columns

for col in cat_cols:
    df.loc[:, col] = df[col].astype(str).str.strip().str.lower()

In [29]:
# Convert 'dt_customer' column to date-time format
if 'dt_customer' in df.columns:
    df['dt_customer'] = pd.to_datetime(df['dt_customer'], errors='coerce')

In [30]:
# Handle outliers using IQR (for numeric columns)
num_cols = df.select_dtypes(include=np.number).columns

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df.loc[:, col] = df[col].clip(lower, upper)

/tmp/ipython-input-3891835183.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[120.5   2.  111.  ...  32.   80.    2. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].clip(lower, upper)
/tmp/ipython-input-3891835183.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[88.  6. 42. ... 24. 61. 21.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].clip(lower, upper)


In [33]:
# Final summary
print("\nFinal shape:", df.shape)
print("Remaining missing values:", df.isnull().sum().sum())


Final shape: (2240, 29)
Remaining missing values: 1324


In [35]:
# columns which still have missing values
missing_cols = df.columns[df.isnull().any()].tolist()
print("Columns still having missing values:\n", missing_cols)

print(df[missing_cols].isnull().sum())

Columns still having missing values:
 ['dt_customer']
dt_customer    1324
dtype: int64


In [36]:
for col in missing_cols:
    print(f"\n--- {col} ---")
    print(df[col].unique()[:10])
    print(df[col].dtype)


--- dt_customer ---
<DatetimeArray>
['2012-04-09 00:00:00', '2014-08-03 00:00:00',                 'NaT',
 '2014-10-02 00:00:00', '2013-09-09 00:00:00', '2013-08-05 00:00:00',
 '2013-06-06 00:00:00', '2012-10-10 00:00:00', '2012-03-11 00:00:00',
 '2012-08-08 00:00:00']
Length: 10, dtype: datetime64[ns]
datetime64[ns]


In [37]:
df.replace([' ', 'NA', 'N/A', 'n/a', 'null', 'Null', 'NaN', 'nan', 'unknown'], np.nan, inplace=True)

In [38]:
df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))

/tmp/ipython-input-3647172377.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))


In [39]:
# Fill remaining missing numeric values and categorical columns
num_cols = df.select_dtypes(include='number').columns
for col in num_cols:
    df[col].fillna(df[col].median(), inplace=True)

cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

/tmp/ipython-input-354378630.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-354378630.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [41]:
print("Remaining missing values:", df.isnull().sum().sum())

Remaining missing values: 0


In [42]:
# Save cleaned data
df.to_csv("Cleaned_Customer_Personality.csv", index=False)
print("\n Cleaned dataset saved successfully!")


 Cleaned dataset saved successfully!
